In [1]:
import sys
import os
import numpy as np
sys.path.append('/Users/iwatobipen/develop/chemoenv/molsg')

In [2]:
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with x3d backend.


In [4]:
from molsg import laplacemesh as lp

In [22]:
# define show mol
def plot_mol(mesh, scalars=np.zeros(1)):
    """Use mayavi to plot a molecule mesh."""
    vertices, faces = mesh
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    fig = mlab.figure(size=(600, 600), bgcolor=(0, 0, 0))
    mesh = mlab.triangular_mesh(x, y, z, faces, scalars=scalars)
    mlab.show()
    return fig, mesh

In [23]:
# load molecule and plot
mol = np.load('data/ampc/npy/actives18.pqr0.50.41.2.npy')
eigs = lp.compute_lb_fem(vertices=mol[0], faces=mol[1], k=100)

In [25]:
f,m = plot_mol(mol, scalars=eigs.phi[:, 1])
f
m

In [26]:
f,m = plot_mol(mol, scalars=eigs.phi[:, 5])
f
m

In [27]:
#calculate descriptor
from scipy.spatial import distance

from molsg.laplacemesh import compute_lb_fem
from molsg.localgeometry import WKS_descriptor
import molsg.covariance as cov
evals = 32

mol_names = sorted(os.listdir('data/ampc/npy/'))
mols = (np.load('data/ampc/npy/{}'.format(m)) for m in mol_names)
eigs = (compute_lb_fem(vertices=m[0], faces=m[1], k=100) for m in mols)
wks = (WKS_descriptor(e, evals=evals) for e in eigs)
covs = np.asarray([cov.covariance_descriptor(w).flatten() for w in wks])

# use first molecule as reference
similarity = distance.cdist(covs[:10], covs,
                            metric='braycurtis')
ranking = np.argsort(similarity)
labels = np.asarray([1 if m[0] == 'a' else 0 for m in mol_names])

In [30]:
covs.shape

(500, 1024)

In [31]:
covs

array([[3.09230117e-05, 2.71652856e-05, 2.37224905e-05, ...,
        1.46243371e-04, 1.51304765e-04, 1.56195547e-04],
       [4.23007865e-05, 3.51307176e-05, 2.88876041e-05, ...,
        8.37931399e-05, 8.68547261e-05, 8.98288537e-05],
       [4.89053334e-05, 4.14805807e-05, 3.47848596e-05, ...,
        8.90841519e-05, 9.20655185e-05, 9.49510127e-05],
       ...,
       [6.53880098e-05, 5.46076646e-05, 4.49903276e-05, ...,
        8.51860637e-05, 8.78986016e-05, 9.05161684e-05],
       [4.03589638e-05, 3.60930158e-05, 3.19959977e-05, ...,
        8.99413803e-05, 9.29632995e-05, 9.59047178e-05],
       [8.57205191e-05, 7.30392856e-05, 6.15872617e-05, ...,
        4.91097210e-05, 5.07646684e-05, 5.23721508e-05]])